In [1]:
import pandas as pd

# Load dataframe
df = pd.read_csv("combined_test_1.csv")

In [2]:
df["Instance n - alpha"] = df.name + "-" + df.alpha.astype(int).astype(str)

In [6]:
df1 = df[["Instance n - alpha", "total_time", "gap", "lower_bound", "upper_bound", "num_hubs", "num_subtour", "node_count", "algorithm", "one_cut", "transformation"]]
# df1['gap'] = df1['gap'].astype(float).map("{:.2%}".format)
df1[["gap", "upper_bound", "lower_bound", "total_time"]] = df1[["gap", "upper_bound", "lower_bound", "total_time"]].apply(lambda x: round(x, 2))
df1[["num_hubs", "num_subtour", "node_count"]] = df1[["num_hubs", "num_subtour", "node_count"]].astype("int32")

df_bf = df1.loc[df1["algorithm"] == " brute_force"][["Instance n - alpha", "total_time"]].rename(columns = {"total_time": "CPU_BF"})

df1_merged = df1.merge(df_bf, how ="left", on = "Instance n - alpha")
col_name = ["Instance n - alpha", "total_time", "gap", "lower_bound", "upper_bound", "num_hubs", "num_subtour", "node_count", "algorithm"]

/var/folders/kp/yc47d3y17093qphx6ccckxg40000gn/T/ipykernel_48092/3320053580.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[["gap", "upper_bound", "lower_bound", "total_time"]] = df1[["gap", "upper_bound", "lower_bound", "total_time"]].apply(lambda x: round(x, 2))
/var/folders/kp/yc47d3y17093qphx6ccckxg40000gn/T/ipykernel_48092/3320053580.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[["num_hubs", "num_subtour", "node_count"]] = df1[["num_hubs", "num_subtour", "node_count"]].astype("int32")

In [8]:
df1_merged

,Instance n - alpha,total_time,gap,lower_bound,upper_bound,num_hubs,num_subtour,node_count,algorithm,one_cut,transformation,CPU_BF
0,att48-3,3600.36,0.43,253638.63,445447.0,5,18714,1146003,bbc,false,false,10.19
1,att48-3,3600.36,0.20,357006.18,445447.0,5,26481,1031143,bbc,false,true,10.19
2,att48-3,3600.68,0.81,85470.81,445447.0,5,27070,788044,bbc,true,false,10.19
3,att48-3,3600.06,0.49,228141.73,445447.0,5,23395,940834,bbc,true,true,10.19
4,att48-5,3600.65,0.69,197512.68,644105.0,5,15873,877581,bbc,false,false,13.50
...,...,...,...,...,...,...,...,...,...,...,...,...
187,ulysses22-3,489.55,-0.00,531.00,531.0,5,143,26254,ilp,false,true,0.25
188,ulysses22-5,3600.40,0.24,540.00,710.0,5,1325,120257,ilp,false,false,0.19
189,ulysses22-5,3600.41,0.18,580.00,710.0,5,533,150659,ilp,false,true,0.19
190,ulysses22-7,3600.22,0.57,363.00,848.0,5,2740,102813,ilp,false,false,0.21


In [9]:
df_bbc = df1_merged.loc[
                        df1_merged.algorithm.str.contains(" bbc") &
                        df1_merged.one_cut.str.contains(" false") &
                        df1_merged.transformation.str.contains(" true")
][df1_merged.columns & col_name]

df_ilp = df1_merged.loc[
                        df1_merged.algorithm.str.contains(" ilp") &
                        df1_merged.one_cut.str.contains(" false") &
                        df1_merged.transformation.str.contains(" false")
][df1_merged.columns & col_name]

/var/folders/kp/yc47d3y17093qphx6ccckxg40000gn/T/ipykernel_48092/574074362.py:5: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  ][df1_merged.columns & col_name]
/var/folders/kp/yc47d3y17093qphx6ccckxg40000gn/T/ipykernel_48092/574074362.py:11: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  ][df1_merged.columns & col_name]


In [10]:
# Get the merged dataframe 
df_merged_res = pd.concat([df_bbc, df_ilp], ignore_index=True)

In [11]:
df_pivot = df_merged_res.pivot_table(index = "Instance n - alpha", columns = "algorithm", values = ["total_time", "gap", "lower_bound", "upper_bound", "num_hubs", "num_subtour", "node_count"])

In [15]:
df_pivot.columns = df_pivot.columns.swaplevel()
df_pivot = df_pivot[sorted(df_pivot.columns)]

In [16]:
df_pivot

algorithm              bbc                                              \
                       gap lower_bound node_count num_hubs num_subtour   
Instance n - alpha                                                       
 att48-3              0.20   357006.18    1031143        5       26481   
 att48-5              0.49   328557.14     987883        5       20402   
 att48-7              0.67   206604.65     765650       47       22667   
 bays29-3             0.00    18877.00       5418        5         114   
 bays29-5             0.13    22379.04     948851        5       10698   
 bays29-7             0.40    15220.75     777466       28       18232   
 burma14-3            0.00      243.00       3297        5          46   
 burma14-5            0.00      305.00    1364387        5        2043   
 burma14-7            0.46      163.10     179328        5        3503   
 dantzig42-3          0.30     3495.98    1461599        5        6077   
 dantzig42-5          0.52     2894.22     963130        5       10844   
 dantzig42-7          0.66     1840.75     709842       16       14621   
 eil51-3              0.00     4423.00     103296        5         573   
 eil51-5              0.31     4652.11    1218161        5        9325   
 eil51-7              0.61     2954.84     614704       51       14371   
 small_instance_10-5  0.00     3025.00          0        6         160   
 small_instance_20-5  0.04    47300.29          0        6        3297   
 swiss42-3            0.00    11569.00       6767        5         174   
 swiss42-5            0.30    12539.48     972898        5       10683   
 swiss42-7            0.55     7775.20     758988       42       18297   
 ulysses22-3          0.14      458.40    1745790        5        7336   
 ulysses22-5          0.34      470.68     896653        5       21417   
 ulysses22-7          0.62      326.00     970171        5       23508   

algorithm                                     ilp                         \
                     total_time upper_bound   gap lower_bound node_count   
Instance n - alpha                                                         
 att48-3                3600.36    445447.0  0.34    294657.0       1843   
 att48-5                3600.06    644105.0  0.60    255280.0        272   
 att48-7                3600.42    618601.0  0.81    155616.0        177   
 bays29-3                  3.89     18877.0  0.05     17857.0      43733   
 bays29-5               3600.35     25700.0  0.35     16825.0      16954   
 bays29-7               3600.35     25482.0  0.66     10182.0      16549   
 burma14-3                 1.70       243.0  0.00       243.0       1523   
 burma14-5              2480.34       305.0  0.00       305.0      21246   
 burma14-7              4163.94       303.0  0.14       253.0     668511   
 dantzig42-3            3600.12      4986.0  0.00      4986.0      25522   
 dantzig42-5            3600.24      5995.0  0.22      4660.0     260589   
 dantzig42-7            3600.06      5457.0  0.45      2833.0     228670   
 eil51-3                 119.88      4423.0  0.16      3699.0        571   
 eil51-5                3600.79      6700.0  0.49      3400.0        108   
 eil51-7                3600.54      7650.0  0.76      2035.0         99   
 small_instance_10-5       8.49      3025.0  0.00      2685.0          0   
 small_instance_20-5    3600.22     49065.0  0.00     45791.0          0   
 swiss42-3                 5.97     11569.0  0.00     11569.0       5596   
 swiss42-5              3600.44     18035.0  0.40     10775.0        896   
 swiss42-7              3600.56     17221.5  0.71      6521.0       1166   
 ulysses22-3            3600.13       531.0  0.05       505.0      66747   
 ulysses22-5            3600.25       710.0  0.24       540.0     120257   
 ulysses22-7            3600.25       848.0  0.57       363.0     102813   

algorithm                                                         
                     num_hubs n